In [1]:
import nltk
from string import punctuation
from nltk.tokenize import RegexpTokenizer, word_tokenize

from unicodedata import normalize
from nltk.stem import RSLPStemmer
from nltk.util import ngrams
import pandas as pd

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')

import time
import pyterrier as pt
pt.init()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [2]:
df_assunto= pd.read_table("../dados-conle-anonimizado-assunto-notnull - dados-conle-anonimizado-assunto-notnull.tsv",sep=";")

In [3]:
arr_assunto = df_assunto.to_numpy()
y,X = arr_assunto[:,0],arr_assunto[:,1]
y = [i.strip() for i in y]

In [4]:
base = pd.read_csv("../base_20230428_douglas.csv", delimiter=',', encoding="utf-8")

In [5]:
base["txtIndexacao"] = base.txtIndexacao.fillna("Em branco")

In [6]:
base

,codProposicao,txtSiglaTipo,numAno,numNumero,txtNome,txtEmenta,txtExplicacaoEmenta,txtIndexacao,txtInteiroTeor
0,16357,PL,1999,1165,PL 1165/1999,"Altera dispositivo da Lei nº 8.987, de 13 de f...",Estabelece que as concessionárias disponibiliz...,"Alteração, Lei das Concessões de Serviços Públ...",Ofício nº 1416 (SF) ...
1,19098,PL,1992,3097,PL 3097/1992,Dispõe sobre a eleição de diretores de fundos ...,NaN,"NORMAS, ELEIÇÃO DIRETA, EMPREGADO, APOSENTADO,...",COMISSÃO DE CONSTITUIÇÃO E JUSTIÇA E DE REDAÇÃ...
2,20464,PL,2000,3927,PL 3927/2000,Altera a composição dos Tribunais Regionais do...,"Altera a composição do TRT da 5ª região, 6ª re...","Alteração, Lei Federal, composição, Tribunal R...","COMISSÃO DE TRABALHO, DE ADMINISTRAÇÃO E SERVI..."
3,20683,PL,1998,4117,PL 4117/1998,Dispõe sobre o acesso a ambientes de uso colet...,NaN,"Autorização, pessoa portadora de deficiência, ...",Câmara dos Deputados\n Departamento ...
4,20857,PL,1998,4395,PL 4395/1998,Estabelece as Diretrizes Nacionais de Defesa C...,NaN,"Fixação, diretrizes, defesa civil, (Sindec), d...",- 1 - \nCOMISSÃO DE CONSTITUIÇÃO E JUSTIÇA E D...
...,...,...,...,...,...,...,...,...,...
57104,2358873,PL,2023,2233,PL 2233/2023,Altera o parágrafo único do art. 71 do Código ...,NaN,Em branco,1 \n \n \n \n \nCÂMARA DOS DEPUTADOS \nPROJET...
57105,2358874,PL,2019,3616,PL 3616/2019,"Altera a Lei nº 9.503, de 23 de setembro de 19...",NaN,Em branco,"Altera a Lei nº 9.503, de 23 de setembro\nde 1..."
57106,2358875,PL,2019,1822,PL 1822/2019,"Altera a Lei nº 11.340, de 7 de agosto de 2006...",NaN,Em branco,"Altera a Lei nº 11.340, de 7 de agosto de\n200..."
57107,2358877,PL,2019,3815,PL 3815/2019,"Altera a Lei nº 7.565, de 19 de dezembro de 19...",NaN,Em branco,"Altera a Lei nº 7.565, de 19 de dezembro\nde ..."


In [7]:
len(base)

57109

In [5]:
def verificar(y,top_n,top_k):
    tamanho = len(top_n) if len(top_n) <20 else top_k
    for i in range(tamanho):
        if len(top_n)>0:
            
            index = int(top_n.loc[i,"docid"])
            nome = base.iloc[index,4]
            if y in nome:
                return 1
        else:
            return 0
    return 0

In [6]:
def avaliacaoRecall(isPreprocess,top_k=20):
    
    quant_encontrado = 0
    quant_relevante =0
    antes = time.time()
    for l,x in zip(y,X):

        tokenized_query3 = x
        if isPreprocess:
            tokenized_query3 = preprocess(x)
        
        top_n = retriever.search(tokenized_query3)
        quant_relevante+=1
        quant_encontrado+=verificar(l,top_n,top_k)
    
    recall = quant_encontrado / quant_relevante
    depois = time.time()
    print("Recall: "+str(recall))
    print("Duração: %f" %(depois - antes))

In [7]:
def avaliacaoRR(isPreprocess):
    quant_relevante =0
    for l,x in zip(y,X):

        tokenized_query3 = x
        if isPreprocess:
            tokenized_query3 = preprocess(x)
        
        top_n = retriever.search(tokenized_query3)
        if len(top_n)>0:
            index = int(top_n.iloc[0,2])
            nome = base.iloc[index,4]
            if l.strip() in nome.strip():
                quant_relevante+=1
    rr = quant_relevante/295
    print("RR: %f" %(rr))

In [8]:
def avaliacaoPrecision(isPreprocess,top_k=20):
    
    quant_encontrado = 0
    soma =0
    for l,x in zip(y,X):

        tokenized_query3 = x
        if isPreprocess:
            tokenized_query3 = preprocess(x)
        
        top_n = retriever.search(tokenized_query3)
    
        
        quant_encontrado=verificar(l,top_n,top_k)
        soma+= (quant_encontrado / top_k)
    
    precision = soma / 295
    
    print("Precision: "+str(precision))

In [9]:
def avaliacaoMAP(isPreprocess,top_k=20):
    
    l_v = list()
    labels_nsL = list()
    for l,x in zip(y,X):

        tokenized_query3 = x
        if isPreprocess:
            tokenized_query3 = preprocess(x)
        
        top_n = retriever.search(tokenized_query3)
    
        l_v.append(l)
        
        if len(top_n)>0:
            labels_nsL.append([base.iloc[int(top_n.iloc[i,4]),1] for i in range(top_k)])
        else:
            labels_nsL.append(list())
            
    denominador = 0
    encontrou=False
    soma=0
    for Y,x in zip(l_v,labels_nsL):
        calc = 0
        quant=0
        if len(x)>0:
            for k in x:
                if str(k).strip() == Y.strip():
                    quant=1
                    encontrou=True
                denominador+=1
                calc=quant/denominador
                if encontrou:
                    calc = calc * 1
                else:
                    calc = calc * 0
                soma+=calc
                encontrou = False
    MAP = soma /295
    print("MAP@ %d: %f" % (top_k,MAP))
            

In [10]:
class Savoy:

    def __removeAllPTAccent(self, old_word):
        word = list(old_word)
        len_word = len(word)-1
        for i in range(len_word, -1, -1):
            if word[i] == 'ä':
                word[i] = 'a'
            if word[i] == 'â':
                word[i] = 'a'
            if word[i] == 'à':
                word[i] = 'a'
            if word[i] == 'á':
                word[i] = 'a'
            if word[i] == 'ã':
                word[i] = 'a'
            if word[i] == 'ê':
                word[i] = 'e'
            if word[i] == 'é':
                word[i] = 'e'
            if word[i] == 'è':
                word[i] = 'e'
            if word[i] == 'ë':
                word[i] = 'e'
            if word[i] == 'ï':
                word[i] = 'i'
            if word[i] == 'î':
                word[i] = 'i'
            if word[i] == 'ì':
                word[i] = 'i'
            if word[i] == 'í':
                word[i] = 'i'
            if word[i] == 'ü':
                word[i] = 'u'
            if word[i] == 'ú':
                word[i] = 'u'
            if word[i] == 'ù':
                word[i] = 'u'
            if word[i] == 'û':
                word[i] = 'u'
            if word[i] == 'ô':
                word[i] = 'o'
            if word[i] == 'ö':
                word[i] = 'o'
            if word[i] == 'ó':
                word[i] = 'o'
            if word[i] == 'ò':
                word[i] = 'o'
            if word[i] == 'õ':
                word[i] = 'o'
            if word[i] == 'ç':
                word[i] = 'c'

        new_word = "".join(word)
        return new_word

    def __finalVowelPortuguese(self, word):
        len_word = len(word)
        if len_word > 3:
            if word[-1] == 'e' or word[-1] == 'a' or word[-1] == 'o':
                word = word[:-1]

        return word

    def __remove_PTsuffix(self, word):
        len_word = len(word)

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'e' and (word[-3] == 'r' or word[-3] == 's' or word[-3] == 'z' or word[-3] == 'l'):
                word = word[:-2]
                return word
        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'n':
                new_word = list(word)
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if (word[-1] == 's' and word[-2] == 'i') and (word[-3] == 'e' or word[-3] == 'é'):
                new_word = list(word)
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'a':
                new_word = list(word)
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'ó':
                new_word = list(word)
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i':
                new_word = list(word)
                new_word[-1] = 'l'
                sing = "".join(new_word)
                return sing

        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'õ':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'ã':
                new_word = list(word)
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 5:
            if word[-1] == 'e' and word[-2] == 't' and word[-3] == 'n' and word[-4] == 'e' and word[-5] == 'm':
                word = word[:-5]
                return word

        if len_word > 2:
            if word[-1] == 's':
                word = word[:-1]

        return word

    def __normFemininPortuguese(self, word):

        len_word = len(word)

        if len_word < 3 or word[-1] != 'a':
            return word

        if len_word > 6:

            if word[-2] == 'h' and word[-3] == 'n' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'c' and word[-3] == 'a' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'r' and word[-3] == 'i' and word[-4] == 'e':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        if len_word > 5:
            if word[-2] == 'n' and word[-3] == 'o':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'r' and word[-3] == 'o':
                word = word[:-1]
                return word

            if word[-2] == 's' and word[-3] == 'o':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 's' and word[-3] == 'e':
                new_word = list(word)
                new_word[-3] = 'ê'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'c' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'v' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'm' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'n':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        return word

    def stem(self, word):
        len_word = len(word)
        if len_word > 2:
            word = self.__remove_PTsuffix(word)
            word = self.__normFemininPortuguese(word)
            word = self.__finalVowelPortuguese(word)
            word = self.__removeAllPTAccent(word)

        return word

In [11]:
class RSLP_S:
    def __plural_reduction(self, word):
        excep = ["lápis","cais","mais","crúcis","biquínis","pois","depois","dois","leis" ]
        excep_s = ["aliás","pires","lápis","cais","mais","mas","menos", "férias","fezes","pêsames","crúcis","gás", "atrás","moisés","através","convés","ês", "país","após","ambas","ambos","messias"]

        len_word = len(word)
        new_word = list(word)

        if len_word >= 3:
            if new_word[-1] == 's' and new_word[-2] == 'n':
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'õ':
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return  sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'ã':
                if word == 'mães':
                    word = word[:-1]
                    return word
                else:
                    new_word[-2] = 'o'
                    sing = "".join(new_word)
                    sing = sing[:-1]
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'a':
                if word != 'cais' and word != 'mais':
                    new_word[-2] = 'l'
                    sing = "".join(new_word)
                    sing = sing[:-1]
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'é':
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'e':
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'ó':
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i':
                if word not in excep:
                    new_word[-1] = 'l'
                    sing = "".join(new_word)
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'l':
                word = word[:-2]
                return word

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'r':
                word = word[:-2]
                return word

            if new_word[-1] == 's':
                if word not in excep_s:
                    word = word[:-1]

        return word

    def stem(self, word):
        word = self.__plural_reduction(word)

        return word

# Config 1

In [12]:
index_path = 'E:\Corpora_em_json\experimentos_pyterrier\config1t'

In [13]:
retriever = pt.BatchRetrieve(index_path,{"vmodel":"BM25","k1":1.5,"b":0.75})

In [14]:
def preprocess(txt):
    txt = str(txt)
    tokenizer = pt.autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()
    terms = tokenizer.getTokens(txt)
    terms = " ".join(terms)
    return terms

## Avaliação

### Recall

In [21]:
avaliacaoRecall(True,5)

Recall: 0.003389830508474576
Duração: 11.637022


In [22]:
avaliacaoRecall(True,10)

Recall: 0.006779661016949152
Duração: 13.802999


In [15]:
avaliacaoRecall(True,20)

Recall: 0.0847457627118644
Duração: 11.442348


### Precision

In [ ]:
avaliacaoPrecision(True,5)

In [ ]:
avaliacaoPrecision(True,10)

In [ ]:
avaliacaoPrecision(True,20)

### MAP

In [ ]:
avaliacaoMAP(True,5)

In [ ]:
avaliacaoMAP(True,10)

In [ ]:
avaliacaoMAP(True,20)

### RR

In [ ]:
avaliacaoRR(True)

# Config 5

In [63]:
index_path = 'E:\Corpora_em_json\experimentos_pyterrier\config5t'

In [64]:
retriever = pt.BatchRetrieve(index_path,{"vmodel":"BM25","k1":1.5,"b":0.75})

In [65]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    terms = " ".join(terms)
    return terms

## Avaliação

### Recall

In [27]:
avaliacaoRecall(True,5)

Recall: 0.24067796610169492
Duração: 10.042999


In [28]:
avaliacaoRecall(True,10)

Recall: 0.3050847457627119
Duração: 9.580893


In [66]:
avaliacaoRecall(True,20)

Recall: 0.3593220338983051
Duração: 8.206036


### Precision

In [ ]:
avaliacaoPrecision(True,5)

In [ ]:
avaliacaoPrecision(True,10)

In [ ]:
avaliacaoPrecision(True,20)

### MAP

In [ ]:
avaliacaoMAP(True,5)

In [ ]:
avaliacaoMAP(True,10)

In [ ]:
avaliacaoMAP(True,20)

### RR

In [ ]:
avaliacaoRR(True)

# Config 8

In [67]:
index_path = 'E:\Corpora_em_json\experimentos_pyterrier\config8t'

In [68]:
retriever = pt.BatchRetrieve(index_path,{"vmodel":"BM25","k1":1.5,"b":0.75})

In [69]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

## Avaliação

### Recall

In [33]:
avaliacaoRecall(True,5)

Recall: 0.3220338983050847
Duração: 11.000994


In [34]:
avaliacaoRecall(True,10)

Recall: 0.39322033898305087
Duração: 10.584998


In [70]:
avaliacaoRecall(True,20)

Recall: 0.4610169491525424
Duração: 9.284023


### Precision

In [36]:
avaliacaoPrecision(True,5)

Precision: 0.06440677966101682


In [37]:
avaliacaoPrecision(True,10)

Precision: 0.039322033898305


In [38]:
avaliacaoPrecision(True,20)

Precision: 0.023050847457627064


### MAP

In [39]:
avaliacaoMAP(True,5)

MAP@ 5: 0.000000


In [40]:
avaliacaoMAP(True,10)

MAP@ 10: 0.000000


In [41]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000000


### RR

In [42]:
avaliacaoRR(True)

RR: 0.159322


# Config 9

In [71]:
index_path = 'E:\Corpora_em_json\experimentos_pyterrier\config9t'

In [72]:
retriever = pt.BatchRetrieve(index_path,{"vmodel":"BM25","k1":1.5,"b":0.75})

In [73]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

## Avaliação

### Recall

In [46]:
avaliacaoRecall(True,5)

Recall: 0.3254237288135593
Duração: 10.267003


In [47]:
avaliacaoRecall(True,10)

Recall: 0.4101694915254237
Duração: 10.110002


In [74]:
avaliacaoRecall(True,20)

Recall: 0.4610169491525424
Duração: 8.924218


### Precision

In [ ]:
avaliacaoPrecision(True,5)

In [ ]:
avaliacaoPrecision(True,10)

In [ ]:
avaliacaoPrecision(True,20)

### MAP

In [ ]:
avaliacaoMAP(True,5)

In [ ]:
avaliacaoMAP(True,10)

In [ ]:
avaliacaoMAP(True,20)

### RR

In [ ]:
avaliacaoRR(True)

# Config 18

In [75]:
index_path = 'E:\Corpora_em_json\experimentos_pyterrier\config18t'

In [76]:
retriever = pt.BatchRetrieve(index_path,{"vmodel":"BM25","k1":1.5,"b":0.75})

In [77]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)

## Avaliação

### Recall

In [52]:
avaliacaoRecall(True,5)

Recall: 0.3559322033898305
Duração: 10.192660


In [53]:
avaliacaoRecall(True,10)

Recall: 0.41694915254237286
Duração: 9.976284


In [78]:
avaliacaoRecall(True,20)

Recall: 0.46779661016949153
Duração: 9.969999


### Precision

In [ ]:
avaliacaoPrecision(True,5)

In [ ]:
avaliacaoPrecision(True,10)

In [ ]:
avaliacaoPrecision(True,20)

### MAP

In [ ]:
avaliacaoMAP(True,5)

In [ ]:
avaliacaoMAP(True,10)

In [ ]:
avaliacaoMAP(True,20)

### RR

In [ ]:
avaliacaoRR(True)

# Config 21

In [79]:
index_path = 'E:\Corpora_em_json\experimentos_pyterrier\config21t'

In [80]:
retriever = pt.BatchRetrieve(index_path,{"vmodel":"BM25","k1":1.5,"b":0.75})

In [81]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)

## Avaliação

### Recall

In [58]:
avaliacaoRecall(True,5)

Recall: 0.3559322033898305
Duração: 9.772276


In [59]:
avaliacaoRecall(True,10)

Recall: 0.41694915254237286
Duração: 9.382999


In [82]:
avaliacaoRecall(True,20)

Recall: 0.4610169491525424
Duração: 9.530033


### Precision

In [ ]:
avaliacaoPrecision(True,5)

In [ ]:
avaliacaoPrecision(True,10)

In [ ]:
avaliacaoPrecision(True,20)

### MAP

In [ ]:
avaliacaoMAP(True,5)

In [ ]:
avaliacaoMAP(True,10)

In [ ]:
avaliacaoMAP(True,20)

### RR

In [ ]:
avaliacaoRR(True)

# Config 22

In [83]:
index_path = 'E:\Corpora_em_json\experimentos_pyterrier\config22t'

In [84]:
retriever = pt.BatchRetrieve(index_path,{"vmodel":"BM25","k1":1.5,"b":0.75})

In [85]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]

    
    return " ".join(terms)

## Avaliação

### Recall

In [ ]:
avaliacaoRecall(True,5)

In [ ]:
avaliacaoRecall(True,10)

In [86]:
avaliacaoRecall(True,20)

Recall: 0.4542372881355932
Duração: 8.481016


### Precision

In [ ]:
avaliacaoPrecision(True,5)

In [ ]:
avaliacaoPrecision(True,10)

In [ ]:
avaliacaoPrecision(True,20)

### MAP

In [ ]:
avaliacaoMAP(True,5)

In [ ]:
avaliacaoMAP(True,10)

In [ ]:
avaliacaoMAP(True,20)

### RR

In [ ]:
avaliacaoRR(True)

# Config 23

In [87]:
index_path = 'E:\Corpora_em_json\experimentos_pyterrier\config23t'

In [88]:
retriever = pt.BatchRetrieve(index_path,{"vmodel":"BM25","k1":1.5,"b":0.75})

In [89]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)

## Avaliação

### Recall

In [ ]:
avaliacaoRecall(True,5)

In [ ]:
avaliacaoRecall(True,10)

In [90]:
avaliacaoRecall(True,20)

Recall: 0.4610169491525424
Duração: 8.921421


### Precision

In [ ]:
avaliacaoPrecision(True,5)

In [ ]:
avaliacaoPrecision(True,10)

In [ ]:
avaliacaoPrecision(True,20)

### MAP

In [ ]:
avaliacaoMAP(True,5)

In [ ]:
avaliacaoMAP(True,10)

In [ ]:
avaliacaoMAP(True,20)

### RR

In [ ]:
avaliacaoRR(True)